In [1]:
!git clone https://github.com/abachaa/MedQuAD.git

Cloning into 'MedQuAD'...
remote: Enumerating objects: 11310, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 11310 (delta 7), reused 5 (delta 5), pack-reused 11300 (from 1)
Receiving objects: 100% (11310/11310), 11.01 MiB | 14.74 MiB/s, done.
Resolving deltas: 100% (6807/6807), done.
Updating files: 100% (11277/11277), done.


In [2]:
!rm -r /content/MedQuAD/10_MPlus_ADAM_QA

In [3]:
!rm -r /content/MedQuAD/11_MPlusDrugs_QA

In [4]:
!rm -r /content/MedQuAD/12_MPlusHerbsSupplements_QA

In [18]:
import xml.etree.ElementTree as ET
import csv
import os
import pandas as pd
import numpy as np

In [5]:
with open('/content/MedQuAD/1_CancerGov_QA/0000001_1.xml', 'r') as f:
  data = f.read()

In [41]:
def parse(filepath):

  with open(filepath, 'r') as f:
    xml_data = f.read()

  # Parse the XML data.
  root = ET.fromstring(xml_data)

  # Extract document-level metadata.
  doc_id = root.attrib.get("id", "")
  doc_source = root.attrib.get("source", "")
  doc_url = root.attrib.get("url", "")
  focus = root.findtext("Focus", default="").strip()

  # Extract UMLS metadata (if available).
  umls_elem = root.find("FocusAnnotations/UMLS")
  if umls_elem is not None:
      cuis = [cui.text.strip() for cui in umls_elem.find("CUIs").findall("CUI")] if umls_elem.find("CUIs") is not None else []
      semantic_types = [st.text.strip() for st in umls_elem.find("SemanticTypes").findall("SemanticType")] if umls_elem.find("SemanticTypes") is not None else []
      semantic_group = umls_elem.findtext("SemanticGroup", default="").strip()
  else:
      cuis = []
      semantic_types = []
      semantic_group = ""

  # Prepare the CSV file for writing.
  csv_filename = f"extracted_data/{focus.split('/')[0]}_{doc_id}.csv"
  with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
      fieldnames = [
          "Document_ID",
          "Document_Source",
          "Document_URL",
          "Focus",
          "UMLS_CUIs",
          "UMLS_SemanticTypes",
          "UMLS_SemanticGroup",
          "QAPair_pid",
          "Question_qid",
          "Question_qtype",
          "Question",
          "Answer"
      ]
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer.writeheader()

      # Iterate over each QAPair element.
      for qa_pair in root.findall("QAPairs/QAPair"):
          pid = qa_pair.attrib.get("pid", "")

          question_elem = qa_pair.find("Question")
          question_text = question_elem.text.strip() if question_elem is not None and question_elem.text else ""
          question_qid = question_elem.attrib.get("qid", "") if question_elem is not None else ""
          question_qtype = question_elem.attrib.get("qtype", "") if question_elem is not None else ""

          answer_elem = qa_pair.find("Answer")
          answer_text = answer_elem.text.strip() if answer_elem is not None and answer_elem.text else ""

          # Create a row that includes both document metadata and the QA pair data.
          row = {
              "Document_ID": doc_id,
              "Document_Source": doc_source,
              "Document_URL": doc_url,
              "Focus": focus,
              "UMLS_CUIs": "; ".join(cuis),
              "UMLS_SemanticTypes": "; ".join(semantic_types),
              "UMLS_SemanticGroup": semantic_group,
              "QAPair_pid": pid,
              "Question_qid": question_qid,
              "Question_qtype": question_qtype,
              "Question": question_text,
              "Answer": answer_text
          }
          writer.writerow(row)

  # print(f"Data extraction complete. CSV file '{csv_filename}' has been created.")


In [42]:
parse('/content/MedQuAD/1_CancerGov_QA/0000001_1.xml')

In [43]:
os.chdir('/content/MedQuAD/')
os.getcwd()

'/content/MedQuAD'

In [44]:
dirs = os.listdir()
dirs.sort()

In [45]:
dirs = dirs[2:-4]

In [46]:
!rm -r /content/MedQuAD/extracted_data/*

In [47]:
for dir in dirs:
  try:
    currpath = os.getcwd()
    all_files = os.listdir(currpath + '/' + dir)
    for f in all_files:
      parse(currpath + '/' + dir + '/' + f)
  except:
    print('currpath', currpath)
    print('dir', dir)
    print('f', f)

In [50]:
os.chdir('/content/MedQuAD/extracted_data')

In [51]:
len(os.listdir())

5493

In [52]:
all_files = os.listdir()
xdf = pd.DataFrame()
for f in all_files:
  xdf = pd.concat([xdf, pd.read_csv(f)])

In [56]:
xdf.reset_index(inplace=True, drop=True)

In [57]:
xdf.head()

,Document_ID,Document_Source,Document_URL,Focus,UMLS_CUIs,UMLS_SemanticTypes,UMLS_SemanticGroup,QAPair_pid,Question_qid,Question_qtype,Question,Answer
0,3472,GARD,https://rarediseases.info.nih.gov/gard/10323/l...,L-arginine:glycine amidinotransferase deficiency,C2675179,T047,Disorders,1,0003472-1,information,What is (are) L-arginine:glycine amidinotransf...,L-arginine:glycine amidinotransferase (AGAT) d...
1,3472,GARD,https://rarediseases.info.nih.gov/gard/10323/l...,L-arginine:glycine amidinotransferase deficiency,C2675179,T047,Disorders,2,0003472-2,symptoms,What are the symptoms of L-arginine:glycine am...,What are the signs and symptoms of L-arginine:...
2,3382,GARD,https://rarediseases.info.nih.gov/gard/6830/ke...,Kernicterus,C0022610,T047,Disorders,1,0003382-1,information,What is (are) Kernicterus ?,Kernicterus is a rare condition that affects t...
3,3382,GARD,https://rarediseases.info.nih.gov/gard/6830/ke...,Kernicterus,C0022610,T047,Disorders,2,0003382-2,symptoms,What are the symptoms of Kernicterus ?,What are the signs and symptoms of Kernicterus...
4,5204,GARD,https://rarediseases.info.nih.gov/gard/10428/p...,Pure autonomic failure,C0393911,T047,Disorders,1,0005204-1,information,What is (are) Pure autonomic failure ?,Pure autonomic failure is characterized by gen...


In [59]:
xdf.to_csv('/content/all_data.csv')